In [1]:
!pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.1/880.1 kB 23.5 MB/s eta 0:00:00


In [2]:
import re
import tensorflow as tf
import numpy as np
import keras
import random
import os
from keras.layers import Dense, Input, TextVectorization, Embedding, Bidirectional,LSTM,Dropout
from keras.models import Sequential
from keras_nlp.layers import TransformerDecoder, TokenAndPositionEmbedding
from keras_nlp.metrics import Perplexity
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from nltk.metrics.distance import edit_distance

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Using TensorFlow backend


In [3]:
raw_texts = ""
#dataset: https://www.kaggle.com/datasets/tunguz/the-pg19-language-modeling-benchmark-dataset
print("Reading from file: ")
def read_data(file):
    file = "/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/"+file+".txt"
    if os.path.isfile(file) == True:
        print(file)
        with open(file, "r") as f:
            raw_text = f.readlines()
        return raw_text

for i in range(50):
    num = random.randint(1,99)
    filename = str(100)+str(num)
    data = read_data(filename)
    #print(type(data))
    if type(data) == list:
        #print(data)
        raw_texts += str(data[100:])


Reading from file: 
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/1004.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10032.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10026.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10046.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10045.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10085.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10067.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10063.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10039.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10035.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10068.txt
/kaggle/input/the-pg19-language-modeling-benchmark-dataset/train/train/10019.txt
/kaggle/i

In [4]:
def clean_data(raw_texts):
    raw_texts = re.sub(r'\n','',raw_texts)
    raw_texts = re.sub(r'\\n','',raw_texts)
    raw_texts = re.sub(r"\'",'',raw_texts)
    raw_texts = re.sub(r'[*]','',raw_texts)
    raw_texts = re.sub(r'\\','',raw_texts)
    raw_texts = re.sub(r',','',raw_texts)
    raw_texts = re.sub(r'[0-9]+','',raw_texts)
    
    return raw_texts

data = clean_data(raw_texts)
data = data.split('.')


In [5]:
#Using Glove Embeddings
file = "/kaggle/input/glove-embeddings/glove.6B.100d.txt" 
with open(file, "r") as f:
    glove_data = f.readlines()

In [6]:
glove_embeddings = dict()
for i in range(len(glove_data)):
    word = glove_data[i].split()[0]
    token = glove_data[i].split()[1:]
    token =  np.asarray([float(t) for t in token])    
    glove_embeddings[word] = token    

In [7]:
max_tokens = 20000
max_length = 50
batch_size=128
vectorizer = TextVectorization(output_sequence_length=max_length+1)
vectorizer.adapt(data)

In [8]:
vocab = vectorizer.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

In [9]:
vocab_size = len(vocab)
vocab_size

53339

In [10]:
embedding_dim = 100
max_tokens = 20000
hits = 0
misses = 0
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 34505 words (18834 misses)


In [11]:
train = data[:int(0.85*len(data))]
test = data[int(0.85*len(data)):]
train = tf.data.Dataset.from_tensor_slices(train).batch(batch_size)
#train = vectorizer(train)
test = tf.data.Dataset.from_tensor_slices(test).batch(batch_size)
#test = vectorizer(test)

In [12]:
def vectorize_dataset(text):
    text = tf.expand_dims(text, -1)
    tokenized_text = vectorizer(text)

    x = tokenized_text[:, :-1]
    y = tokenized_text[:, 1:]
    return x,y

train = train.map(vectorize_dataset)
train = train.prefetch(tf.data.AUTOTUNE)
#train = train.batch(1000)

test = test.map(vectorize_dataset)
test = test.prefetch(tf.data.AUTOTUNE)
#test = test.batch(1000)

In [13]:
def model():
    input_layer = Input(shape=(max_length,), dtype = tf.int32)
    
    embedding_layer = Embedding(
        vocab_size,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        trainable=False,
    )(input_layer)
    
    #embedding_layer = TokenAndPositionEmbedding(vocab_size, max_length, 256)(input_layer)
    bilstm_layer = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)
    dropout_layer = Dropout(0.2)(bilstm_layer)
    bilstm_dense_layer = Dense(128)(dropout_layer)
    
    decoder_layer = TransformerDecoder(
        intermediate_dim = 128,
        num_heads = 4,
        dropout=0.2,
        activation="relu")(embedding_layer)
    output_layer = Dense(vocab_size,
                         activation="softmax")(decoder_layer)

    model = keras.Model(inputs = input_layer, outputs = output_layer)
    return model
    

In [14]:
model = model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 100)           5333900   
                                                                 
 transformer_decoder (Transf  (None, 50, 100)          66628     
 ormerDecoder)                                                   
                                                                 
 dense_3 (Dense)             (None, 50, 53339)         5387239   
                                                                 
Total params: 10,787,767
Trainable params: 5,453,867
Non-trainable params: 5,333,900
_________________________________________________________________


In [15]:
model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=[Perplexity(), 'accuracy']
    )

In [16]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, verbose=1)
model.fit(train, validation_data = test, epochs=10, verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/10
727/727 [==============================] - 262s 350ms/step - loss: 2.5028 - perplexity: 12.2161 - accuracy: 0.7056 - val_loss: 1.8899 - val_perplexity: 6.6187 - val_accuracy: 0.7492 - lr: 0.0010
Epoch 2/10
727/727 [==============================] - 269s 370ms/step - loss: 1.9814 - perplexity: 7.2526 - accuracy: 0.7152 - val_loss: 1.8509 - val_perplexity: 6.3654 - val_accuracy: 0.7507 - lr: 0.0010
Epoch 3/10
727/727 [==============================] - 271s 373ms/step - loss: 1.9103 - perplexity: 6.7550 - accuracy: 0.7165 - val_loss: 1.8341 - val_perplexity: 6.2592 - val_accuracy: 0.7514 - lr: 0.0010
Epoch 4/10
727/727 [==============================] - 271s 372ms/step - loss: 1.8635 - perplexity: 6.4460 - accuracy: 0.7172 - val_loss: 1.8366 - val_perplexity: 6.2753 - val_accuracy: 0.7514 - lr: 0.0010
Epoch 5/10
727/727 [==============================] - 271s 373ms/step - loss: 1.8285 - perplexity: 6.2249 - accuracy: 0.7176 - val_loss: 1.8357 - val_perplexity: 6.2695 - val_accu

In [17]:
def generate_text(text,num_words):
    #text = "It is a wonderful"
    if num_words > 0:
        for i in range(num_words):
            text = text.lower()
            text +=" "
            seq_length = len(text.split()) - 1
            text_vec = vectorizer([text])[:, :-1]
            predictions = model.predict([text_vec],verbose=0)
            logits,index = tf.math.top_k(predictions[0][seq_length], k=20, sorted=True)
            index = np.asarray(index).astype("int32")
            for i in range(5):
                ran = random.randint(0,len(index))
                word = list(word_index.keys())[list(word_index.values()).index(ran)]
                #print(word)
                if word != '[UNK]' and word.isalpha() == True:
                    text += word
                    #print(word)
                    num_words-=1
                    #print(num_words)
                    break
            #print(text)
        generate_text(text,num_words)
    elif num_words == 0:
         print(text)
        


In [28]:
texts = ["It is a wonderful",
        "I have not been very",
        "That was not",
        "He must not have been"]
for text in texts:
    print(generate_text(text,5))

it is a wonderful for in of to the
None
i have not been very you is was had was
None
that was not had a in a of
None
he must not have been with the in as that
None
